Concatenate individual

In [39]:
def concatenate(start, goal, individual):
    individual_full=individual.copy()

    individual_full.insert(0,start)
    individual_full.append(goal)

    return individual_full

Crossover Methods:
1. Single point crossover
2. Uniform crossover
3. Multipoint crossover

In [40]:
def crossover_sp(parent1, parent2, co_rate, gene_len):
    import numpy as np

    # Perform one-point crossover
    child1 = parent1.copy() 
    child2 = parent2.copy()
    if np.random.rand() < co_rate:
        crossover_point = np.random.randint(1, gene_len)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

In [41]:
def crossover_uniform(parent1, parent2, co_rate, _):
    """
    Uniform Crossover:
    Randomly select genes from both parents to create offspring.
    """
    import numpy as np

    shorter_parent = parent1 if len(parent1) < len(parent2) else parent2
    longer_parent = parent2 if len(parent1) < len(parent2) else parent1

    child1 = shorter_parent.copy()
    child2 = shorter_parent.copy()

    for i in range(len(shorter_parent)):
        if np.random.rand() < co_rate:
            child1[i], child2[i] = child2[i], child1[i]

    for i in range(len(shorter_parent), len(longer_parent)):
        if np.random.rand() < co_rate:
            if longer_parent == parent1:
                child1.append(longer_parent[i])
            else:
                child2.append(longer_parent[i])

    return child1, child2


In [42]:
def crossover_mp(parent1, parent2, co_rate, gene_len):
    import numpy as np

    # Perform multipoint crossover
    child1 = parent1.copy() 
    child2 = parent2.copy()
    
    if np.random.rand() < co_rate:
        crossover_points = sorted(np.random.choice(range(gene_len), 4, replace=False))

        # switch genes between parents in the crossover regions
        for i in range(0, len(crossover_points), 2):
            start = crossover_points[i]
            end = crossover_points[i+1]
            child1[start:end], child2[start:end] = child2[start:end], child1[start:end]

    return child1, child2


Mutation Methods:
1. Point mutation (random)
2. Swap mutation
3. Inversion mutation

In [43]:
def mutate_random(individual, m_rate, gene_len, environment, dimensions):

    import numpy as np
    
    # Perform random mutation
    if np.random.rand() < m_rate:
        
        if dimensions==3:
            index = np.random.randint(0, len(individual))
            x = np.random.randint(low=0, high=environment.shape[0])
            y = np.random.randint(low=0, high=environment.shape[1])
            z = np.random.randint(low=0, high=environment.shape[2])

            if environment[x, y, z] != 0:
                individual[index] = (x, y, z)
        else: 
            index = np.random.randint(0, len(individual))
            x = np.random.randint(low=0, high=environment.shape[1])
            y = np.random.randint(low=0, high=environment.shape[0])

            if environment[y,x] != 0:
                individual[index] = (x, y)

    return individual

In [44]:
def mutate_swap(individual, m_rate, gene_len, environment, dimensions):
    import numpy as np
    
    # Perform swap mutation
    if np.random.rand() < m_rate:
        if dimensions == 3:
            # check the length of the individual
            if len(individual) > 1:
                # select two distinct indices
                index1, index2 = np.random.choice(len(individual), 2, replace=False)
                # swap the two genes
                individual[index1], individual[index2] = individual[index2], individual[index1]
        else:
            # check the length of the individual
            if len(individual) > 1:
                # select two distinct indices
                index1, index2 = np.random.choice(len(individual), 2, replace=False)
                # swap the two genes
                individual[index1], individual[index2] = individual[index2], individual[index1]
    
    return individual


In [45]:
def mutate_inversion(individual, m_rate, gene_len, environment, dimensions):
    import numpy as np
    
    # Perform inversion mutation
    if np.random.rand() < m_rate:
        if len(individual) > 2:
            index1, index2 = np.random.choice(len(individual), 2, replace=False)
            if index2 < index1:
                index1, index2 = index2, index1
            individual[index1:index2+1] = individual[index1:index2+1][::-1]
            
    return individual


Post-Processing (Attempt all permuations  : 2^(n-2))

In [46]:
def get_permutations(lst):
    import itertools

    n = len(lst)
    perms = [[lst[0]] + list(subset) + [lst[-1]] for r in range(n-2) for subset in itertools.combinations(lst[1:-1], r)]
    return [x for i, x in enumerate(perms) if x not in perms[:i]]

lst = [(1, 2), (3, 4), (5, 6), (2,3), (4,5)]
result = get_permutations(lst)
print(result)
print(len(result))

[[(1, 2), (4, 5)], [(1, 2), (3, 4), (4, 5)], [(1, 2), (5, 6), (4, 5)], [(1, 2), (2, 3), (4, 5)], [(1, 2), (3, 4), (5, 6), (4, 5)], [(1, 2), (3, 4), (2, 3), (4, 5)], [(1, 2), (5, 6), (2, 3), (4, 5)]]
7


Termination Conditions:
1. Number of generations
2. Time limit
3. Stagnation

In [47]:
def termination_check(current_gen, current_time, average_fitness,NUM_GENERATIONS, TIME_LIMIT_GA, STAGNATION_NUM, method):
    flag=0

    if method=='generations':
        if current_gen==NUM_GENERATIONS:
            flag=1
    elif method=='time':
        if current_time>TIME_LIMIT_GA:
            flag=1
    elif method=='stagnation':
        if len(average_fitness)>=STAGNATION_NUM:
            flag=check_values(average_fitness, STAGNATION_NUM)

    return flag

In [48]:
def check_values(average_fitness, STAGNATION_NUM):
    """
    Returns True if none of the x values in lst after the nth last element
    are greater than the nth last x value, False otherwise.
    """
    last_x = average_fitness[-STAGNATION_NUM]
    for x in average_fitness[-STAGNATION_NUM+1:]:
        if x > last_x:
            return 0
    return 1